# A notebook for experiments

### Load data


In [4]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyreadstat

data_2022 = pd.read_hdf('data/data_2022.h5', key='data_2022')

<class 'pandas.core.frame.DataFrame'>
Index: 3676029 entries, 0 to 3676028
Data columns (total 64 columns):
 #   Column       Dtype   
---  ------       -----   
 0   DOB_YY       int32   
 1   DOB_MM       category
 2   BFACIL       category
 3   F_BFACIL     category
 4   MAGE_IMPFLG  category
 5   MAGE_REPFLG  category
 6   MAGER        category
 7   MAGER14      category
 8   MAGER9       category
 9   MBSTATE_REC  category
 10  RESTATUS     category
 11  MRACE31      category
 12  MRACE6       category
 13  MRACE15      category
 14  MRACEIMP     category
 15  MHISPX       category
 16  MHISP_R      category
 17  F_MHISP      category
 18  MRACEHISP    category
 19  MAR_P        category
 20  DMAR         category
 21  MAR_IMP      category
 22  F_MAR_P      category
 23  MEDUC        category
 24  F_MEDUC      category
 25  FAGERPT_FLG  category
 26  FAGECOMB     category
 27  FAGEREC11    category
 28  FRACE31      category
 29  FRACE6       category
 30  FRACE15      category
 

In [6]:
data_2022["CA_DOWN"].value_counts()

CA_DOWN
N    3666667
U       7501
P       1043
C        818
Name: count, dtype: int64

In [ ]:
us_residents_only = data_2022[data_2022["RESTATUS"] != 4]

us_residents_only["CA_DOWN"].value_counts()

CA_DOWN
N    3658502
U       7399
P       1039
C        818
Name: count, dtype: int64

In [11]:
foreign = data_2022[data_2022["RESTATUS"] == 4]

foreign_ds = foreign["CA_DOWN"].value_counts()
foreign_ds

CA_DOWN
N    8165
U     102
P       4
C       0
Name: count, dtype: int64

In [12]:
race_recode_31 = foreign["MRACE31"].value_counts()
race_recode_31

MRACE31
1     7628
2      318
4      257
3       22
13      13
6       11
5        8
10       8
8        2
18       1
15       1
23       1
7        1
28       0
27       0
22       0
30       0
26       0
25       0
24       0
29       0
16       0
21       0
20       0
19       0
17       0
14       0
12       0
11       0
9        0
31       0
Name: count, dtype: int64

In [ ]:
race_recode_6 = foreign["MRACE6"].value_counts()
race_recode_6

MRACE6
1    7628
2     318
4     257
6      38
3      22
5       8
Name: count, dtype: int64

In [ ]:
race_recode_15 = foreign["MRACE15"].value_counts()
race_recode_15

MRACE15
1     7628
2      318
5       95
4       69
10      39
15      38
6       28
3       22
9       10
7        9
8        7
14       5
12       2
11       1
13       0
Name: count, dtype: int64

In [ ]:
m_hisp_origin = foreign["MHISPX"].value_counts()
m_hisp_origin

MHISPX
1    6507
0    1181
9     312
4     131
6     108
5      13
2      11
3       8
Name: count, dtype: int64

Export

In [ ]:
# pyreadstat.write_sav(data_2022, "data/data_2022.sav")

# pyreadstat.write_dta(data_2022, "data/data_2022.dta")

# data_2022.to_hdf("data/data_2022.h5", key="data_2022", format="table")

# data_2022.to_csv("data/data_2022.csv", index=False)